In [15]:
! pip install rank_bm25

Defaulting to user installation because normal site-packages is not writeable


In [16]:
#!pip install rank_bm25
! git clone https://github.com/cr-nlp/project1-2023.git

import urllib.request as re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import numpy as np
from collections import defaultdict
import nltk
nltk.download('stopwords')
nltk.download('punkt')


def loadNFCorpus():
	dir = "./project1-2023/"
	filename = dir +"dev.docs"

	dicDoc={}
	with open(filename) as file:
		lines = file.readlines()
	for line in lines:
		tabLine = line.split('\t')
		#print(tabLine)
		key = tabLine[0]
		value = tabLine[1]
		#print(value)
		dicDoc[key] = value
	filename = dir + "dev.all.queries"
	dicReq={}
	with open(filename) as file:
		lines = file.readlines()
	for line in lines:
		tabLine = line.split('\t')
		key = tabLine[0]
		value = tabLine[1]
		dicReq[key] = value
	filename = dir + "dev.2-1-0.qrel"
	dicReqDoc=defaultdict(dict)
	with open(filename) as file:
		lines = file.readlines()
	for line in lines:
		tabLine = line.strip().split('\t')
		req = tabLine[0]
		doc = tabLine[2]
		score = int(tabLine[3])
		dicReqDoc[req][doc]=score

	return dicDoc, dicReq, dicReqDoc

def text2TokenList(text):
	stopword = stopwords.words('english')
	#print("LEN DE STOPWORD=",len(stopword))
	word_tokens = word_tokenize(text.lower())
	word_tokens_without_stops = [word for word in word_tokens if word not in stopword and len(word)>2]
	return word_tokens_without_stops



fatal: destination path 'project1-2023' already exists and is not an empty directory.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\siney\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\siney\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [17]:
dicDoc, dicReq, dicReqDoc = loadNFCorpus()

## Pretrained word2vec

In [18]:
import gensim

In [21]:
model = "C:/Users/siney/Downloads/wikitovec.txt"
model1 = gensim.models.KeyedVectors.load_word2vec_format(model, binary=False)

In [2]:

model2 = "PubMed-w2v.bin"
model2 = gensim.models.KeyedVectors.load_word2vec_format(model2, binary=True)

In [ ]:
#Please enter one of the above pretrained word2vec to test
word_vectors = model2

In [22]:
import numpy as np
def get_vector(text):
    try:
        vector =word_vectors.get_vector(text)
        return np.mean(vector)
    except KeyError:
        #print(f"{text} not found.")
        return 0

In [23]:
docsToKeep=[]
reqsToKeep=[]
dicReqDocToKeep=defaultdict(dict)

#150
ndcgTop=5
#print("ndcgTop=",ndcgTop,"nbDocsToKeep=",nbDocsToKeep)
startDoc = 0
endDoc = 3196
i=startDoc
for reqId in dicReqDoc:
  if i > (endDoc - startDoc) :  #nbDocsToKeep:
    break
  for docId in dicReqDoc[reqId]:
    dicReqDocToKeep[reqId][docId] = dicReqDoc[reqId][docId]
    docsToKeep.append(docId)
    i = i + 1
  reqsToKeep.append(reqId)
docsToKeep = list(set(docsToKeep))

In [24]:
import rank_bm25
from rank_bm25 import BM25Okapi

corpusDocTokenList = []
corpusReqTokenList = {}
corpusDocName=[]
corpusDicoDocName={}
i = 0
for k in docsToKeep:
  docTokenList = [get_vector(i) for i in text2TokenList(dicDoc[k])]
  corpusDocTokenList.append(docTokenList)
  corpusDocName.append(k)
  corpusDicoDocName[k] = i
  i = i + 1

#print("reqs...")
corpusReqName=[]
corpusDicoReqName={}
i = 0
for k in reqsToKeep:
  reqTokenList = [get_vector(i) for i in text2TokenList(dicReq[k])]
  corpusReqTokenList[k] = reqTokenList
  corpusReqName.append(k)
  corpusDicoReqName[k] = i
  i = i + 1

nih-aarp not found.
ptrend not found.
ptrend not found.
ptrend not found.
ptrend not found.
inter-conversion not found.
erucin not found.
erucin not found.
erucin not found.
erucin not found.
erucin not found.
erucin not found.
folin-ciocalteu not found.
hplc-pda-ms not found.
flavan not found.
hydroxycinnamates not found.
hydroxycinnamates not found.
phenylpyridine not found.
primer-probe not found.
dimethylthiazol not found.
diphenyltetrazolium not found.
transwell not found.
xenografted not found.
transwell not found.
cells/field not found.
cells/field not found.
ecmatrix not found.
nodules/lung not found.
nodules/lung not found.
g/mice not found.
g/mice not found.
antimetastatic not found.
hexaphosphate not found.
n-formyl-met-leu-phe not found.
fortimel not found.
often-suggested not found.
poultry/fish not found.
poultry/fish not found.
pg/g not found.
pg/g not found.
pg/g not found.
hexabromocyclododecane not found.
submicron-sized not found.
non-stimulatory not found.
icrom not

In [25]:
bm25 = BM25Okapi(corpusDocTokenList)

ndcgCumul=0
corpusReqVec={}
ndcgBM25Cumul=0
nbReq=0

from sklearn.metrics import ndcg_score
for req in corpusReqTokenList:
  j=0
  reqTokenList = corpusReqTokenList[req]
  doc_scores = bm25.get_scores(reqTokenList)
  trueDocs = np.zeros(len(corpusDocTokenList))

  for docId in corpusDicoDocName:
    if req in dicReqDocToKeep:
      if docId in dicReqDocToKeep[req]:
        #get position docId
        posDocId = corpusDicoDocName[docId]
        trueDocs[posDocId] = dicReqDocToKeep[req][docId]
        #print("TOKEEP=",docId)
        #print(trueDocs)
  ndcgBM25Cumul = ndcgBM25Cumul + ndcg_score([trueDocs], [doc_scores],k=ndcgTop)
  nbReq = nbReq + 1
ndcgBM25Cumul = ndcgBM25Cumul / nbReq
print("ndcg bm25=",ndcgBM25Cumul)

ndcg bm25= 0.4515476354629547
